In [1]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import src.functions as src


fs=500
freq_bands={'delta' : [0,4],
            'theta' : [4,8],
            'alpha' : [8, 13],
            'beta' : [13,35],
            'high' : [35, 45],
            'all': [1.5,45],
            'low': [1.5,12]}


data = {}
labels = {}
file= [item for item in os.listdir('data') if item.endswith('set')]

for file_index, file_name in enumerate(file):
    path_ = os.path.join( 'data', file_name )
    dados = mne.read_epochs_eeglab(path_)
    channel_names = dados.info['ch_names']
    subject_id = file_name[:3]
    
    # Extract labels
    Corr_labels = len(dados['proCorr','antiCorr'].get_data())
    Err_labels = len(dados['proErr','antiErr','nogoErr'].get_data())
    labels[subject_id] = np.array([0] * Corr_labels + [1] * Err_labels)
    
    # Initialize channel data dictionary
    channel_data = {}
    
    # Iterate over 'proCorr' and 'antiCorr' events
    for epoch_values in dados['proCorr','antiCorr'].get_data():
        for channel_index, channel_values in enumerate(epoch_values):
            channel_name = channel_names[channel_index]
            if channel_name not in channel_data:
                channel_data[channel_name] = []
            channel_data[channel_name].append(list(channel_values))
    
    # Iterate over 'proErr', 'antiErr', and 'nogoErr' events
    for epoch_values in dados['proErr','antiErr','nogoErr'].get_data():
        for channel_index, channel_values in enumerate(epoch_values):
            channel_name = channel_names[channel_index]
            if channel_name not in channel_data:
                channel_data[channel_name] = []
            channel_data[channel_name].append(list(channel_values))
    
    data[subject_id] = channel_data


Extracting parameters from c:\Users\User\Documents\GitHub\Tese\data\P01.set...
Not setting metadata
228 matching events found
No baseline correction applied


C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:24: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  dados = mne.read_epochs_eeglab(path_)


0 projection items activated
Ready.


C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:29: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  Corr_labels = len(dados['proCorr','antiCorr'].get_data())
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:30: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  Err_labels = len(dados['proErr','antiErr','nogoErr'].get_data())
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for epoch_values in dados['proCorr','antiCorr'].get_data():


Extracting parameters from c:\Users\User\Documents\GitHub\Tese\data\P02.set...


...

C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:45: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for epoch_values in dados['proErr','antiErr','nogoErr'].get_data():
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:24: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  dados = mne.read_epochs_eeglab(path_)


Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated
Ready.


C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:29: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  Corr_labels = len(dados['proCorr','antiCorr'].get_data())
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:30: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  Err_labels = len(dados['proErr','antiErr','nogoErr'].get_data())
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for epoch_values in dados['proCorr','antiCorr'].get_data():


Extracting parameters from c:\Users\User\Documents\GitHub\Tese\data\P19.set...
Not setting metadata
111 matching events found
No baseline correction applied
0 projection items activated
Ready.


C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:45: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for epoch_values in dados['proErr','antiErr','nogoErr'].get_data():
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:24: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  dados = mne.read_epochs_eeglab(path_)
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:29: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  Corr_labels = len(dados['proCorr','antiCorr'].get_data())
C:\Users\User\AppData\Local\Temp\ipykernel_26944\752335729.py:30: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  Err_labels = len(dados['proErr','antiErr','

In [2]:
def data_pre_post(data):
    pre_response={}
    post_response={}
    for subject_id, dados in data.items():
        channel_data_pre={}
        channel_data_post={}
        for channel_name, values in dados.items():
            values_pre=[]
            values_post=[]
            for i in range(len(values)):
                
                index= int(len(values[i])/2)
                values_pre.append(values[i][0:index])
                values_post.append(values[i][index:-1])
            channel_data_pre[channel_name]= values_pre
            channel_data_post[channel_name]= values_post
            
        pre_response[subject_id] = channel_data_pre #(14,60,n_events,250)
        post_response[subject_id] = channel_data_post
    return pre_response, post_response

pre_response, post_response = data_pre_post(data)

In [22]:
labels['P01']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1])

In [23]:
a=[]
b=[]
for subject_id in labels.keys():
    for i in labels[subject_id]:
        if i==0:
            a.append(i)
        elif i==1:
            b.append(i)
print(len(a))
print(len(b))

2615
168


In [3]:
f_pre,S_pre= src.getpsd(pre_response,fs)    # S_pre ->(14,60,n_events,n_points)
f_post,S_post= src.getpsd(post_response,fs)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_spectral_py.py:589: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_spectral_py.py:589: UserWarning: nperseg = 256 is greater than input length  = 249, using nperseg = 249
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


In [4]:
pre_data_per_channel= src.getdataperchannel(S_pre) #(subject_id,60,n_events,n_points)
post_data_per_channel= src.getdataperchannel(S_post)

### FCZ cluster 

In [16]:
data1= src.extend_list(src.feature('fcz_features','.','.','theta','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post, f_post))
data2= src.extend_list(src.feature('fcz_features','.','.','theta','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
data3= src.extend_list(src.feature('fcz_features','.','.','theta','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
data4= src.extend_list(src.feature('fcz_features','.','.','theta','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
data5= src.extend_list(src.feature('fcz_features','.','.','theta','other',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
data6= src.extend_list(src.feature('fcz_features','.','.','theta','other',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
data7= src.extend_list(src.feature('fcz_features','.','.','theta','theta',post_data_per_channel,pre_data_per_channel,freq_bands,f_post,f_pre))

### All

In [6]:
data8={}
data9={}
data10={}
data11={}
data12={}
data13={}
data14={}
data15={}
data16={}
for channel_name in post_data_per_channel['P01'].keys():    
    feature8= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature9= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature10= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature11= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature12= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','theta',post_data_per_channel,pre_data_per_channel,freq_bands,f_post,f_pre))
    feature13= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','delta',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature14= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','alpha',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature15= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','delta',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature16= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','alpha',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    data8[channel_name]=feature8
    data9[channel_name]=feature9
    data10[channel_name]=feature10
    data11[channel_name]=feature11
    data12[channel_name]=feature12
    data13[channel_name]=feature13
    data14[channel_name]=feature14
    data15[channel_name]=feature15
    data16[channel_name]=feature16

### Midfrontal

In [7]:
data17={}
data18={}
for channel_name in post_data_per_channel['P01'].keys():
    feature17= src.extend_list(src.feature('midfrontal_features',channel_name,channel_name,'theta','other',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature18= src.extend_list(src.feature('midfrontal_features',channel_name,channel_name,'theta','other',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    data17[channel_name]=feature17
    data18[channel_name]=feature18

### Low

In [8]:
data19={}
data20={}
data21={}
data22={}
data23={}
data24={}
data25={}
data26={}
data27={}
data28={}
data29={}
data30={}
for channel_name in post_data_per_channel[subject_id].keys():        
    
    feature19= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature20= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature21= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature22= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature23= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature24= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature25= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature26= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature27= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature28= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature29= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature30= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    data19[channel_name]=feature19
    data20[channel_name]=feature20
    data21[channel_name]=feature21
    data22[channel_name]=feature22
    data23[channel_name]=feature23
    data24[channel_name]=feature24
    data25[channel_name]=feature25
    data26[channel_name]=feature26
    data27[channel_name]=feature27
    data28[channel_name]=feature28
    data29[channel_name]=feature29
    data30[channel_name]=feature30

### Temporal

In [9]:
data31={}
data32={}
FCZ_Cluster=['FZ','FC1','FCZ','FC2','CZ']
PE_Cluster=['CPZ','P1','PZ','P2','POZ']
for subject_id,values in post_response.items():
    mean_all70_160_values = []
    mean_all200_500_values = []
    for i in range(len(next(iter(values.values())))):
        all70_160_values = []
        all200_500_values = []
        for channel_name, channel_values in values.items():   
            if channel_name in FCZ_Cluster:         
                all70_160_values.append(np.mean(channel_values[i][35:81]))
            elif channel_name in PE_Cluster:
                all200_500_values.append(np.mean(channel_values[i][100:-1]))
            else:
                continue

        mean_all70_160_values.append(np.mean(all70_160_values))
        mean_all200_500_values.append(np.mean(all200_500_values))
    data31[subject_id]= mean_all70_160_values
    data32[subject_id]= mean_all200_500_values

data31=src.extend_list(data31)
data32=src.extend_list(data32)

In [10]:
data33 = {}
data34 = {}
# Iterate over each subject in the pre_response2 dictionary
for subject_id, values in pre_response.items():
    # Iterate over each channel in the subject's data
    for channel_name, channel_values in values.items():
        # If the channel is not yet in the feature dictionaries, initialize it
        if channel_name not in data33:
            data33[channel_name] = []
        if channel_name not in data34:
            data34[channel_name] = []
        
        # Iterate over each event in the channel values
        for i in range(len(channel_values)):
            # Calculate the mean values for the specified ranges
            mean_500_250 = np.mean(channel_values[i][0:126])
            mean_250_0 = np.mean(channel_values[i][125:-1])
            
            # Append the mean values to the corresponding channel's list in the feature dictionaries
            data33[channel_name].append(mean_500_250)
            data34[channel_name].append(mean_250_0)


In [11]:
data35 = {}
data36 = {}
# Iterate over each subject in the pre_response2 dictionary
for subject_id, values in post_response.items():
    # Iterate over each channel in the subject's data
    for channel_name, channel_values in values.items():
        # If the channel is not yet in the feature dictionaries, initialize it
        if channel_name not in data35:
            data35[channel_name] = []
        if channel_name not in data36:
            data36[channel_name] = []
        
        # Iterate over each event in the channel values
        for i in range(len(channel_values)):
            # Calculate the mean values for the specified ranges
            mean_0_250 = np.mean(channel_values[i][0:126])
            mean_250_500 = np.mean(channel_values[i][125:-1])
            
            # Append the mean values to the corresponding channel's list in the feature dictionaries
            data35[channel_name].append(mean_0_250)
            data36[channel_name].append(mean_250_500)

### Saving Features

In [17]:
import pandas as pd

dicts=[]
for feature_index in range(1,37):
    feature_corr_key = 'data{}'.format(feature_index)
    dicts.append(globals()[feature_corr_key])



combined_df = pd.DataFrame()
# Iterate through each dictionary and its index
for index, feature_dict in enumerate(dicts, start=1):
    # Convert the dictionary to a DataFrame
    print(index)
    df = pd.DataFrame(feature_dict)
    # Rename the columns to include the iteration number
    df.columns = [f'{index}_{col}' for col in df.columns]
    # Concatenate the DataFrame to the combined DataFrame
    combined_df = pd.concat([combined_df, df], axis=1)

combined_df['labels'] = src.extend_list(labels)
print(combined_df)
combined_df.to_csv('dataset1_features.csv', index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
           1_0         2_0       3_0        4_0       5_0       6_0       7_0  \
0     0.861052    6.670514  3.576613  35.416911  0.452551  1.794788  0.187845   
1     3.200801   21.233493  3.033360  23.153352  0.824023  1.248317  1.548954   
2     3.197585   58.889374  3.758197  42.532463  1.252747  1.267166  1.177922   
3     3.519466   67.307223  3.066174  36.797555  1.510801  1.073128  1.981933   
4     2.256458   44.046793  1.332020  15.853042  0.979666  0.730604  1.618276   
...        ...         ...       ...        ...       ...       ...       ...   
2778  2.856935   20.805650  0.776932  33.618486  1.113980  1.997012  1.041733   
2779  2.078781   16.945361  1.035114   8.283552  1.125025  0.598368  2.026725   
2780  3.197488  121.819241  2.739000  25.821557  3.454946  2.133419  2.465279   
2781  3.177710   35.945131  1.085080  14.167883  2.430133  0.935494  2.882115   
2782  2.26